# API (參考https://github.com/mlouielu/twstock)

In [2]:
import datetime
import urllib.parse
from collections import namedtuple
import time
import random
import requests
from codes import codes

try:
    from json.decoder import JSONDecodeError
except ImportError:
    JSONDecodeError = ValueError


headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}


TWSE_BASE_URL = 'http://www.twse.com.tw/'
TPEX_BASE_URL = 'http://www.tpex.org.tw/'
DATATUPLE = namedtuple('Data', ['date', 'capacity', 'turnover', 'open',
                                'high', 'low', 'close', 'change', 'transaction'])


class BaseFetcher(object):
    def fetch(self, year, month, sid, retry):
        pass

    def _convert_date(self, date):
        """Convert '106/05/01' to '2017/05/01'"""
        return '/'.join([str(int(date.split('/')[0]) + 1911)] + date.split('/')[1:])

    def _make_datatuple(self, data):
        pass

    def purify(self, original_data):
        pass


class TWSEFetcher(BaseFetcher):
    REPORT_URL = urllib.parse.urljoin(
        TWSE_BASE_URL, 'exchangeReport/STOCK_DAY')

    def __init__(self):
        pass

    def fetch(self, year: int, month: int, sid: str, retry: int=5):
        params = {'date': '%d%02d01' % (year, month), 'stockNo': sid}
        for retry_i in range(retry):
            r = requests.get(self.REPORT_URL, params=params, headers=headers)
            try:
                data = r.json()
                #print(data)
            except JSONDecodeError:
                print('JSONDecodeError:{}, retry={}'.format(sid, retry_i))
                continue
            else:
                break
        else:
            # Fail in all retries
            data = {'stat': '', 'data': []}

        if data['stat'] == 'OK':
            data['data'] = self.purify(data)
        else:
            data['data'] = []
        time.sleep(random.uniform(2,5))
        return data

    def _make_datatuple(self, data):
        data[0] = datetime.datetime.strptime(
            self._convert_date(data[0]), '%Y/%m/%d')
        data[1] = int(data[1].replace(',', ''))
        data[2] = int(data[2].replace(',', ''))
        data[3] = None if data[3] == '--' else float(data[3].replace(',', ''))
        data[4] = None if data[4] == '--' else float(data[4].replace(',', ''))
        data[5] = None if data[5] == '--' else float(data[5].replace(',', ''))
        data[6] = None if data[6] == '--' else float(data[6].replace(',', ''))
        # +/-/X表示漲/跌/不比價
        data[7] = float(0.0 if data[7].replace(',', '') ==
                        'X0.00' else data[7].replace(',', ''))
        data[8] = int(data[8].replace(',', ''))
        return DATATUPLE(*data)

    def purify(self, original_data):
        return [self._make_datatuple(d) for d in original_data['data']]



class TPEXFetcher(BaseFetcher):
    REPORT_URL = urllib.parse.urljoin(TPEX_BASE_URL,
                                      'web/stock/aftertrading/daily_trading_info/st43_result.php')

    def __init__(self):
        pass

    def fetch(self, year: int, month: int, sid: str, retry: int=5):
        params = {'d': '%d/%d' % (year - 1911, month), 'stkno': sid}
        for retry_i in range(retry):
            r = requests.get(self.REPORT_URL, params=params)
            try:
                data = r.json()
            except JSONDecodeError:
                time.sleep(2)
                continue
            else:
                break
        else:
            # Fail in all retries
            data = {'aaData': []}

        data['data'] = []
        if data['aaData']:
            data['data'] = self.purify(data)
        
        return data

    def _convert_date(self, date):
        """Convert '106/05/01' to '2017/05/01'"""
        return '/'.join([str(int(date.split('/')[0]) + 1911)] + date.split('/')[1:])

    def _make_datatuple(self, data):
        data[0] = datetime.datetime.strptime(self._convert_date(data[0].replace('＊', '')),
                                             '%Y/%m/%d')
        data[1] = int(data[1].replace(',', '')) * 1000
        data[2] = int(data[2].replace(',', '')) * 1000
        data[3] = None if data[3] == '--' else float(data[3].replace(',', ''))
        data[4] = None if data[4] == '--' else float(data[4].replace(',', ''))
        data[5] = None if data[5] == '--' else float(data[5].replace(',', ''))
        data[6] = None if data[6] == '--' else float(data[6].replace(',', ''))
        data[7] = float(data[7].replace(',', ''))
        data[8] = int(data[8].replace(',', ''))
        return DATATUPLE(*data)

    def purify(self, original_data):
        return [self._make_datatuple(d) for d in original_data['aaData']]


class Stock(object):

    def __init__(self, sid: str, initial_fetch: bool=True):
        self.sid = sid
        self.fetcher = TWSEFetcher(
        ) if codes[sid].market == '上市' else TPEXFetcher()
        self.raw_data = []
        self.data = []

        # Init data
        if initial_fetch:
            self.fetch_31()

    def _month_year_iter(self, start_month, start_year, end_month, end_year):
        ym_start = 12 * start_year + start_month - 1
        ym_end = 12 * end_year + end_month
        for ym in range(ym_start, ym_end):
            y, m = divmod(ym, 12)
            yield y, m + 1

    def fetch(self, year: int, month: int):
        """Fetch year month data"""
        self.raw_data = [self.fetcher.fetch(year, month, self.sid)]
        self.data = self.raw_data[0]['data']
        return self.data

    def fetch_from(self, year: int, month: int):
        """Fetch data from year, month to current year month data"""
        self.raw_data = []
        self.data = []
        today = datetime.datetime.today()
        for year, month in self._month_year_iter(month, year, today.month, today.year):
            self.raw_data.append(self.fetcher.fetch(year, month, self.sid))
            self.data.extend(self.raw_data[-1]['data'])
        return self.data

    def fetch_31(self):
        """Fetch 31 days data"""
        today = datetime.datetime.today()
        before = today - datetime.timedelta(days=60)
        self.fetch_from(before.year, before.month)
        self.data = self.data[-31:]
        return self.data

    @property
    def date(self):
        return [d.date for d in self.data]

    @property
    def capacity(self):
        return [d.capacity for d in self.data]

    @property
    def turnover(self):
        return [d.turnover for d in self.data]

    @property
    def price(self):
        return [d.close for d in self.data]

    @property
    def high(self):
        return [d.high for d in self.data]

    @property
    def low(self):
        return [d.low for d in self.data]

    @property
    def open(self):
        return [d.open for d in self.data]

    @property
    def close(self):
        return [d.close for d in self.data]

    @property
    def change(self):
        return [d.change for d in self.data]

    @property
    def transaction(self):
        return [d.transaction for d in self.data]

# 取得某年月股價 (fetch)

In [3]:
stockcode='2330'
stock=Stock(stockcode)
stock.fetch(2020,4)

[Data(date=datetime.datetime(2020, 4, 1, 0, 0), capacity=47572034, turnover=13022612818, open=276.5, high=276.5, low=271.5, close=271.5, change=-2.5, transaction=19580),
 Data(date=datetime.datetime(2020, 4, 6, 0, 0), capacity=59712754, turnover=16324198154, open=273.0, high=275.5, low=270.0, close=275.5, change=4.0, transaction=19971),
 Data(date=datetime.datetime(2020, 4, 7, 0, 0), capacity=48887346, turnover=13817936851, open=283.5, high=284.0, low=280.5, close=283.0, change=7.5, transaction=24281),
 Data(date=datetime.datetime(2020, 4, 8, 0, 0), capacity=38698826, turnover=11016972354, open=285.0, high=285.5, low=283.0, close=285.0, change=2.0, transaction=19126),
 Data(date=datetime.datetime(2020, 4, 9, 0, 0), capacity=29276430, turnover=8346209654, open=287.5, high=288.0, low=282.5, close=283.0, change=-2.0, transaction=15271),
 Data(date=datetime.datetime(2020, 4, 10, 0, 0), capacity=28206858, turnover=7894277586, open=280.0, high=282.0, low=279.0, close=279.5, change=-3.5, tran

# 取得最近31天股價

In [84]:
#最近31天
stock=Stock('2330')
stock.fetch_31
stock.data

[Data(date=datetime.datetime(2020, 4, 7, 0, 0), capacity=48887346, turnover=13817936851, open=283.5, high=284.0, low=280.5, close=283.0, change=7.5, transaction=24281),
 Data(date=datetime.datetime(2020, 4, 8, 0, 0), capacity=38698826, turnover=11016972354, open=285.0, high=285.5, low=283.0, close=285.0, change=2.0, transaction=19126),
 Data(date=datetime.datetime(2020, 4, 9, 0, 0), capacity=29276430, turnover=8346209654, open=287.5, high=288.0, low=282.5, close=283.0, change=-2.0, transaction=15271),
 Data(date=datetime.datetime(2020, 4, 10, 0, 0), capacity=28206858, turnover=7894277586, open=280.0, high=282.0, low=279.0, close=279.5, change=-3.5, transaction=15833),
 Data(date=datetime.datetime(2020, 4, 13, 0, 0), capacity=22196596, turnover=6202532380, open=278.5, high=281.5, low=278.5, close=278.5, change=-1.0, transaction=12809),
 Data(date=datetime.datetime(2020, 4, 14, 0, 0), capacity=50196688, turnover=14268689661, open=281.0, high=285.5, low=280.0, close=285.0, change=6.5, tra

# 列印出日期，股價, 成交量

In [3]:
for _date, _price, _turnover in zip(stock.date, stock.price, stock.turnover):
    #print(_date, _price, _turnover)
    print('{:%Y-%m-%d} {} {}'.format(_date, _price, _turnover))

2020-05-04 295.0 21341931218
2020-05-05 295.5 7104774785
2020-05-06 296.0 10524838077
2020-05-07 297.5 8321170642
2020-05-08 297.5 10619091212
2020-05-11 301.0 9729431840
2020-05-12 295.0 15553012596
2020-05-13 297.0 13619538824
2020-05-14 293.0 10119787160
2020-05-15 298.0 13170466660
2020-05-18 290.0 28216346996
2020-05-19 291.5 15527338138
2020-05-20 294.0 10772280152


# update stock code (csv)

In [28]:
from codes import fetch
fetch.__update_codes()

# list all the codes

In [47]:
for k,v in codes.items():
    print (k,v.name, v.market)

702038 中光電台新97購01 上櫃
702039 群聯台新97購01 上櫃
704875 穩懋中信95購01 上櫃
705010 穩懋中信95購02 上櫃
705503 保瑞國泰98購01 上櫃
705580 昂寶富邦95購01 上櫃
705608 威剛元富95購01 上櫃
705684 建榮統一96購01 上櫃
705685 建榮統一96購02 上櫃
705730 群聯元富96購01 上櫃
705741 富喬統一96購01 上櫃
705775 宜鼎群益96購01 上櫃
705776 原相群益96購01 上櫃
705788 緯軟元大96購01 上櫃
705803 原相富邦97購01 上櫃
705805 環球晶富邦97購01 上櫃
705847 環球晶統一99購01 上櫃
705848 禾昌元富96購01 上櫃
705869 穩懋元大96購01 上櫃
705904 六角富邦97購01 上櫃
705905 昇達科富邦97購01 上櫃
705924 大江元富96購01 上櫃
706054 穩懋中信95購03 上櫃
706056 穩懋元富95購01 上櫃
706108 威剛富邦97購01 上櫃
706109 雙鴻富邦97購01 上櫃
706112 原相元大95購01 上櫃
706130 雙美群益95購01 上櫃
706133 鈺太國票95購02 上櫃
706134 穩懋富邦95購01 上櫃
706164 博智元大95購01 上櫃
706167 三貝德國票95購01 上櫃
706171 鐿鈦國票95購01 上櫃
706173 神盾國泰95購01 上櫃
706175 譜瑞國泰95購01 上櫃
706194 博智國票95購01 上櫃
706199 昇達科國泰95購01 上櫃
706200 保瑞國泰95購02 上櫃
706256 神盾富邦96購01 上櫃
706267 合晶元大96購01 上櫃
706296 穩懋統一96購01 上櫃
706302 三貝德國票96購01 上櫃
706357 網家元富96購01 上櫃
706373 雙鴻國泰96購01 上櫃
706383 群聯群益96購01 上櫃
706393 昇達科元大96購01 上櫃
706408 昂寶元大96購01 上櫃
706410 信昌電元大96購01 上櫃
706415 緯軟元大96購02 上櫃
706417 雙鴻元大

707701 環球晶元富96購01 上櫃
707702 鈊象台新98購01 上櫃
707703 M31凱基96購01 上櫃
707704 金可凱基97購01 上櫃
707705 金居凱基96購01 上櫃
707706 精華凱基96購02 上櫃
707707 環球晶凱基96購04 上櫃
707708 聯亞凱基96購01 上櫃
707709 鐿鈦凱基97購01 上櫃
707710 大田國票97購01 上櫃
707711 六角國票98購01 上櫃
707712 昂寶國票97購02 上櫃
707713 晶焱國票97購01 上櫃
707714 群聯國票97購01 上櫃
707715 大江國泰98購01 上櫃
707716 晶焱國泰98購01 上櫃
707717 穩懋國泰97購01 上櫃
707718 良維永豐96購01 上櫃
707719 威剛永豐96購01 上櫃
707720 宣德永豐97購01 上櫃
707721 鈊象永豐96購03 上櫃
707722 三貝德群益96購01 上櫃
707723 宜鼎群益96購02 上櫃
707724 長科*群益96購01 上櫃
707725 閎康群益97購01 上櫃
707726 晟德群益96購01 上櫃
707727 鈊象群益99購02 上櫃
707728 鈊象康和96購03 上櫃
707729 鈊象凱基98購01 上櫃
707730 穩懋元富96購01 上櫃
707731 環球晶亞東97購01 上櫃
707732 凡甲永豐97購01 上櫃
707733 大宇資永豐96購02 上櫃
707734 元太國票97購01 上櫃
707735 台燿國票97購01 上櫃
707736 信驊國票97購01 上櫃
707737 新普國票97購01 上櫃
707738 聖暉國泰96購01 上櫃
707739 網家國泰97購01 上櫃
707740 金可兆豐96購01 上櫃
707741 胡連康和96購01 上櫃
707742 佳邦群益96購01 上櫃
707743 旺矽群益97購01 上櫃
707744 新普群益98購01 上櫃
707745 網龍群益98購01 上櫃
707746 由田元大96購01 上櫃
707747 信昌電元大97購01 上櫃
707748 神盾元大99購01 上櫃
707749 精華元大97購01 上櫃
707750 穩懋元大9

708264 宣德國票97購02 上櫃
708265 聚積國票97購02 上櫃
708266 頎邦國票97購01 上櫃
708267 M31國泰98購01 上櫃
708268 聯亞國泰98購01 上櫃
708269 頎邦國泰97購01 上櫃
708270 昂寶富邦98購01 上櫃
708271 原相富邦98購01 上櫃
708272 頎邦富邦98購01 上櫃
708273 力旺元大96購02 上櫃
708274 中美晶元大96購05 上櫃
708275 六角元大98購01 上櫃
708276 台半元大96購02 上櫃
708277 金居元大97購01 上櫃
708278 勝麗元大96購03 上櫃
708279 頎邦元大96購02 上櫃
708280 頎邦元大96購03 上櫃
708281 頎邦元大97購01 上櫃
708282 兆利群益98購01 上櫃
708283 頎邦群益97購01 上櫃
708284 頎邦群益97購02 上櫃
708285 頎邦群益98購01 上櫃
708286 三貝德康和96購01 上櫃
708287 兆利康和9C購01 上櫃
708288 富喬康和96購01 上櫃
708289 杰力群益96購01 上櫃
708290 威剛群益97購01 上櫃
708291 環宇群益98購01 上櫃
708292 M31國票97購02 上櫃
708293 合晶國票98購01 上櫃
708294 良維國票97購03 上櫃
708295 昂寶國票97購04 上櫃
708296 威剛國票97購02 上櫃
708297 茂達國票97購02 上櫃
708298 原相國票98購02 上櫃
708299 穩懋國票97購02 上櫃
708300 群聯日盛96購01 上櫃
708301 優群永豐97購01 上櫃
708302 穩懋永豐96購02 上櫃
708303 伍豐中信9A購01 上櫃
708304 同致中信98購02 上櫃
708305 宜鼎中信9A購01 上櫃
708306 閎康統一99購01 上櫃
708307 頎邦統一96購02 上櫃
708308 大中凱基98購01 上櫃
708309 大江凱基97購01 上櫃
708310 兆利凱基98購01 上櫃
708311 迅得凱基97購01 上櫃
708312 佳邦凱基98購01 上櫃
708313 金居凱基98購01

709102 三貝德群益98購01 上櫃
709103 中美晶群益98購01 上櫃
709104 良維群益98購01 上櫃
709105 宣德群益98購01 上櫃
709106 統新群益98購01 上櫃
709107 晶焱群益98購02 上櫃
709108 金居國票99購01 上櫃
709109 統新國票98購01 上櫃
709110 聯亞國票99購01 上櫃
709111 VHQ永豐98購01 上櫃
709112 大宇資永豐98購01 上櫃
709113 大學光永豐9A購01 上櫃
709114 波若威永豐99購01 上櫃
709115 金可永豐98購01 上櫃
709116 環球晶永豐98購01 上櫃
709117 大江凱基98購05 上櫃
709118 元太凱基98購01 上櫃
709119 台燿凱基98購02 上櫃
709120 同致凱基98購03 上櫃
709121 合晶凱基98購03 上櫃
709122 宜鼎凱基98購03 上櫃
709123 昇達科凱基98購02 上櫃
709124 信昌電凱基98購04 上櫃
709125 原相凱基98購04 上櫃
709126 勝麗凱基98購02 上櫃
709127 群聯凱基98購03 上櫃
709128 廣明凱基98購04 上櫃
709129 璟德凱基98購02 上櫃
709130 優群凱基9A購02 上櫃
709131 穩懋凱基98購03 上櫃
709132 穩懋凱基98購04 上櫃
709133 頎邦凱基98購01 上櫃
709134 環球晶統一98購01 上櫃
709135 原相凱基99購01 上櫃
709136 群聯凱基98購04 上櫃
709137 碩禾凱基9A購01 上櫃
709138 M31元大99購02 上櫃
709139 鈊象元大99購04 上櫃
709140 泰博國票99購01 上櫃
709141 神盾國泰9A購01 上櫃
709142 宣德富邦9A購01 上櫃
709143 環球晶富邦9A購01 上櫃
709144 信昌電亞東99購01 上櫃
709145 大江永豐98購01 上櫃
709146 同致永豐98購01 上櫃
709147 合晶永豐98購02 上櫃
709148 良維永豐98購01 上櫃
709149 東隆興永豐98購01 上櫃
709150 南光永豐98購01 上櫃
709151

709884 穩懋元大99購05 上櫃
709885 信驊兆豐99購01 上櫃
709886 中美晶群益9A購01 上櫃
709887 神盾群益9B購02 上櫃
709888 聯亞群益99購01 上櫃
709889 台燿元富9C購01 上櫃
709890 原相統一9A購01 上櫃
709891 大宇資凱基9B購01 上櫃
709892 宇峻凱基9B購01 上櫃
709893 信驊凱基9A購01 上櫃
709894 統新凱基9B購01 上櫃
709895 順達凱基99購01 上櫃
709896 雙美凱基9B購01 上櫃
709897 雙鴻元大99購06 上櫃
709898 穩懋元大99購06 上櫃
709899 穩懋元大99購07 上櫃
709900 力致永豐99購03 上櫃
709901 光頡永豐99購01 上櫃
709902 新鉅科永豐99購01 上櫃
709903 精測永豐9A購01 上櫃
709904 璟德永豐99購01 上櫃
709905 環球晶永豐99購03 上櫃
709906 兆利國票99購01 上櫃
709907 金居國票9A購01 上櫃
709908 信驊國票9A購02 上櫃
709909 兆利國泰9A購01 上櫃
709910 宏捷科國泰9B購01 上櫃
709911 精測國泰9C購01 上櫃
709912 威剛富邦9C購01 上櫃
709913 M31群益99購01 上櫃
709914 中美晶群益9B購01 上櫃
709915 台燿群益99購02 上櫃
709916 合晶群益9A購01 上櫃
709917 緯軟群益9A購01 上櫃
709918 璟德群益99購01 上櫃
709919 環球晶群益99購02 上櫃
709920 原相日盛9B購01 上櫃
709921 雍智國泰9C購01 上櫃
709922 璟德國泰9A購01 上櫃
709923 VHQ群益99購01 上櫃
709924 中美晶群益99購03 上櫃
709925 精測群益99購01 上櫃
709926 精測富邦9A購01 上櫃
709927 聯亞國票9A購01 上櫃
709928 穩懋國票99購03 上櫃
709929 大江永豐99購01 上櫃
709930 精測永豐99購01 上櫃
709931 建榮統一9B購03 上櫃
709932 中美晶凱基99購07 上櫃
709933 碩禾

...

062870 亞德客兆豐9C購01 上市
062871 大成鋼兆豐9B購01 上市
062872 華通兆豐9C購01 上市
062873 國巨兆豐9C購01 上市
062874 微星兆豐9B購01 上市
062875 瑞昱兆豐9C購01 上市
062876 全新兆豐9B購01 上市
062877 可成兆豐9C購01 上市
062878 宏達電兆豐9B購01 上市
062879 美食兆豐9B購01 上市
06287P 臺股指國票97售01 上市
062880 大立光兆豐9B購02 上市
062881 緯創兆豐9C購01 上市
062882 新日興兆豐9C購01 上市
062883 智易兆豐9C購01 上市
062884 康控兆豐9C購01 上市
062885 臻鼎兆豐9C購01 上市
062886 中興電群益9B購01 上市
062887 台積電群益9C購05 上市
062888 漢唐群益9B購01 上市
062889 可成群益9C購01 上市
06288P 臺股指國票97售02 上市
062890 祥碩群益9B購04 上市
062891 台汽電群益9B購01 上市
062892 臺指反元大02購13 上市
062893 元金2X元大9C購01 上市
062894 道瓊銀元大9B購02 上市
062895 道瓊銀元大9C購01 上市
062896 中興電元大9B購01 上市
062897 上銀元大9B購06 上市
062898 華通元大9B購05 上市
062899 鴻海元大9C購04 上市
06289P 臺股指國票97售03 上市
062900 台積電元大9B購04 上市
062901 台積電元大9B購05 上市
062902 智邦元大9B購07 上市
062903 燿華元大9B購04 上市
062904 台光電元大9B購06 上市
062905 聯發科元大9B購13 上市
062906 全新元大9B購03 上市
062907 可成元大9C購03 上市
062908 晶豪科元大9B購03 上市
062909 大立光元大9C購04 上市
06290P 臺股指國票97售04 上市
062910 亞光元大9B購01 上市
062911 聯詠元大9B購03 上市
062912 新日興元大9B購01 上市
062913 創意元大9B購02 上市
062914 嘉澤元大9B購0

06388P 鴻海兆豐97售04 上市
06389P 聯發科兆豐97售01 上市
06390P 美律國票97售03 上市
06391P 上銀中信97售03 上市
06392P T50反1元大9A售01 上市
06393P 台積電元大97售27 上市
06394P 台積電元大97售28 上市
06395P 台積電元大97售29 上市
06396P 新日興康和97售01 上市
06397P T50反1凱基9A售03 上市
06398P 禾伸堂凱基97售02 上市
06399P 台表科凱基97售05 上市
06400P 鴻海元富97售12 上市
06401P 鴻海元富97售13 上市
06402P 鴻海元富97售14 上市
06403P 鴻海元富97售15 上市
06404P 鴻海元富97售16 上市
06405P 億光元富97售02 上市
06406P 可成元富97售04 上市
06407P 創意元富97售02 上市
06408P 聯茂元富97售02 上市
06409P 康友元富97售03 上市
06410P 祥碩統一97售01 上市
06411P 晶相光永豐97售03 上市
06412P 南亞科日盛97售01 上市
06413P 聯發科凱基97售11 上市
06414P 矽力凱基97售03 上市
06415P 台積電群益97售07 上市
06416P 大立光群益97售06 上市
06417P 台積電兆豐97售03 上市
06418P T50反1統一9A售02 上市
06419P 國巨統一97售09 上市
06420P 大立光統一98售06 上市
06421P 嘉澤統一97售02 上市
06422P 勤誠統一98售01 上市
06423P 亞光台新97售01 上市
06424P 大成鋼永豐97售01 上市
06425P TPK永豐97售02 上市
06426P T50正2元大9B售01 上市
06427P 鴻海元大97售13 上市
06428P 國巨元大97售17 上市
06429P 大立光元大97售15 上市
06430P 大立光元大97售16 上市
06431P 新日興元大97售05 上市
06432P 上銀群益98售01 上市
06433P 大立光群益99售01 上市
06434P 超眾群益98售02 上市
06435P 臺股指群益98售02 上市
06436P 

07071P 立積元富99售01 上市
07072P 元油2X凱基9C售01 上市
07073P 臺股指中信9B售02 上市
07074P FBVIX凱基9A售01 上市
07075P FBVIX凱基9C售04 上市
07076P 鴻海凱基9A售08 上市
07077P 緯穎凱基99售01 上市
07078P 鴻海群益99售02 上市
07079P 台積電群益99售06 上市
07080P 健策群益9A售01 上市
07081P 南電群益9A售01 上市
07082P 美律元大99售06 上市
07083P 台積電統一9A售06 上市
07084P S&P正2元富9B售02 上市
07085P 超眾元富9C售01 上市
07086P 力成元富9C售01 上市
07087P 聯發科台新99售02 上市
07088P 訊芯永豐99售02 上市
07089P 臺指2X中信01售01 上市
07090P 台灣50凱基01售01 上市
07091P SGBR2X凱基9A售02 上市
07092P 上銀凱基99售03 上市
07093P 台積電凱基99售07 上市
07094P 華通國票9A售03 上市
07095P 台灣50元大9C售03 上市
07096P 元石油元大9C售05 上市
07097P 元油2X元大9C售09 上市
07098P 台積電元大99售18 上市
07099P 台積電元大99售19 上市
07100P 華航元大99售01 上市
07101P 台郡元大99售06 上市
07102P 南電元大99售03 上市
07103P 南電元大99售04 上市
07104P 聯發科元富99售04 上市
07105P 大毅元富99售02 上市
07106P 國巨統一9B售05 上市
07107P 臺股指統一99售07 上市
07108P FBVIX台新99售02 上市
07109P 聯發科中信9B售02 上市
07110P 台灣50群益9B售03 上市
07111P 台積電群益99售08 上市
07112P 聯發科群益99售05 上市
07113P 台積電兆豐99售03 上市
07114P 旺宏兆豐9A售01 上市
07115P T50反1日盛9C售02 上市
07116P 台灣50麥證9A售02 上市
07117P SGBR2X凱基9C售01 上市
07118P 台積

07631P 臺股指凱基9A售12 上市
07632P 臺股指凱基9A售14 上市
07633P 台灣50元大9A售18 上市
07634P 金像電元大9A售01 上市
07635P 世芯永豐9A售05 上市
07636P 鴻海國票9A售05 上市
07637P 國巨國票9A售08 上市
07638P 長榮航國票9A售01 上市
07639P 緯穎國票9A售01 上市
07640P 國巨群益9A售07 上市
07641P 上銀凱基9A售05 上市
07642P 華通凱基9A售05 上市
07643P 禾伸堂凱基9A售02 上市
07644P 南電凱基9A售03 上市
07645P 華通元大9A售06 上市
07646P 世芯元大9A售04 上市
07647P 健策國票9A售04 上市
07648P 祥碩國票9B售01 上市
07649P 金像電永豐9A售02 上市
07650P 台灣50中信9A售01 上市
07651P 道瓊指中信01售01 上市
07652P 臺股指中信9A售07 上市
07653P 華新科群益9A售05 上市
07654P 台灣50凱基01售08 上市
07655P 台灣50凱基01售09 上市
07656P 國巨凱基9A售15 上市
07657P 技嘉凱基9A售03 上市
07658P 國泰金凱基9A售01 上市
07659P 中租凱基9A售02 上市
07660P 聯茂凱基9A售03 上市
07661P 台灣50兆豐9A售03 上市
07662P 台積電兆豐9A售03 上市
07663P 台灣50台新01售03 上市
07664P 台灣50元大9C售30 上市
07665P 台灣50元大9C售31 上市
07666P 台灣50元大9C售32 上市
07667P 亞德客元大9A售05 上市
07668P 旺宏元大9A售10 上市
07669P 華新科元大9A售14 上市
07670P 茂林元大9A售02 上市
07671P 超眾元大9A售04 上市
07672P 南電元大9A售05 上市
07673P 電子類元大9A售01 上市
07674P 金融類元大9A售01 上市
07675P 全新永豐9A售02 上市
07676P 華新科永豐9A售02 上市
07677P 祥碩永豐9A售01 上市
07678P 大立光群益9A售03 上市
07679

00732 國泰RMB短期報酬 上市
00733 富邦臺灣中小 上市
00735 國泰臺韓科技 上市
00736 國泰新興市場 上市
00737 國泰AI+ROBO 上市
00738U 元大道瓊白銀 上市
00739 元大MSCI A股 上市
00742 新光內需收益 上市
00743 國泰中國A150 上市
00752 中信中國50 上市
00753L 中信中國50正2 上市
00757 統一FANG+ 上市
00762 元大全球AI 上市
00763U 街口道瓊銅 上市
00766L 台新MSCI中國正2 上市
00770 國泰北美科技 上市
00771 元大US高息特別股 上市
00774B 新光中國政金綠債 上市
00774C 新光中政金綠債+R 上市
00775B 新光投等債15+ 上市
00776 新光ICE美國權值 上市
00783 富邦中証500 上市
008201 BP上證50 上市
00830 國泰費城半導體 上市
00850 元大臺灣ESG永續 上市
00851 台新全球AI 上市
00852L 國泰美國道瓊正2 上市
00861 元大全球未來通訊 上市
00865B 國泰US短期公債 上市
00866 新光SHILLER CAPE 上市
00875 國泰網路資安 上市
020000 富邦特選蘋果N 上市
020002 元富新中國N 上市
020004 兆豐電菁英30N 上市
020006 永昌中小300N 上市
020007 凱基臺灣500N 上市
020008 元大特股高息N 上市
020011 統一微波高息20N 上市
020012 富邦行動通訊N 上市
020015 統一MSCI美低波N 上市
020016 統一MSCI美科技N 上市
02001L 富邦蘋果正二N 上市
02001R 富邦蘋果反一N 上市
9103 美德醫療-DR 上市
910322 康師傅-DR 上市
910482 聖馬丁-DR 上市
9105 泰金寶-DR 上市
910861 神州-DR 上市
9110 越南控-DR 上市
911608 明輝-DR 上市
911616 杜康-DR 上市
911622 泰聚亨-DR 上市
911868 同方友友-DR 上市
912000 晨訊科-DR 上市
912398 友佳-DR 上市
9136 巨騰-DR 上市
9188 精熙-D

# get one of the stock codes details

In [48]:
_code='2330'

print(codes[_code])
print('CodesName of {} : {}'.format (_code, codes[_code].name))

StockCodeInfo(type='股票', code='2330', name='台積電', ISIN='TW0002330008', start='1994/09/05', market='上市', group='半導體業', CFI='ESVUFR')
CodesName of 2330 : 台積電


# code info to dict

In [27]:
dict(codes['2330']._asdict())

{'type': '股票',
 'code': '2330',
 'name': '台積電',
 'ISIN': 'TW0002330008',
 'start': '1994/09/05',
 'market': '上市',
 'group': '半導體業',
 'CFI': 'ESVUFR'}

In [5]:
for _data in stock.data:
    print(_data._fields)
    print(type(_data._fields))

('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction')
<class 'tuple'>
('date', 'capacity', 'turnover', 'open', 'high

# stock trade info to dict (for mongoDB)

In [26]:
stockDict={}
stockList=[]

for _data in stock.data:
    stockDict['code'] =stockcode
    stockDict['dailyinfo']=dict(_data._asdict())
    print(stockDict)

2020-05-04 00:00:00
{'code': '2330', 'dailyinfo': {'date': datetime.datetime(2020, 5, 4, 0, 0), 'capacity': 72252861, 'turnover': 21341931218, 'open': 294.5, 'high': 296.5, 'low': 294.0, 'close': 295.0, 'change': -9.5, 'transaction': 37736}}
2020-05-05 00:00:00
{'code': '2330', 'dailyinfo': {'date': datetime.datetime(2020, 5, 5, 0, 0), 'capacity': 23988405, 'turnover': 7104774785, 'open': 296.5, 'high': 298.0, 'low': 295.0, 'close': 295.5, 'change': 0.5, 'transaction': 13569}}
2020-05-06 00:00:00
{'code': '2330', 'dailyinfo': {'date': datetime.datetime(2020, 5, 6, 0, 0), 'capacity': 35704479, 'turnover': 10524838077, 'open': 294.5, 'high': 296.0, 'low': 292.5, 'close': 296.0, 'change': 0.5, 'transaction': 16046}}
2020-05-07 00:00:00
{'code': '2330', 'dailyinfo': {'date': datetime.datetime(2020, 5, 7, 0, 0), 'capacity': 28017198, 'turnover': 8321170642, 'open': 294.5, 'high': 299.0, 'low': 294.5, 'close': 297.5, 'change': 1.5, 'transaction': 13149}}
2020-05-08 00:00:00
{'code': '2330', 

In [17]:
stockDict={}
stockList=[]
for _data in stock.data:
    stockDict['code'] =stockcode
    stockList.append(dict(_data._asdict())) 
    yearmonth='{:%Y-%m}'.format(_data.date)
stockDict['yearmonth']=yearmonth    
stockDict['dailyinfo']=stockList
print(stockDict)

{'code': '2330', 'yearmonth': '2020-05', 'dailyinfo': [{'date': datetime.datetime(2020, 5, 4, 0, 0), 'capacity': 72252861, 'turnover': 21341931218, 'open': 294.5, 'high': 296.5, 'low': 294.0, 'close': 295.0, 'change': -9.5, 'transaction': 37736}, {'date': datetime.datetime(2020, 5, 5, 0, 0), 'capacity': 23988405, 'turnover': 7104774785, 'open': 296.5, 'high': 298.0, 'low': 295.0, 'close': 295.5, 'change': 0.5, 'transaction': 13569}, {'date': datetime.datetime(2020, 5, 6, 0, 0), 'capacity': 35704479, 'turnover': 10524838077, 'open': 294.5, 'high': 296.0, 'low': 292.5, 'close': 296.0, 'change': 0.5, 'transaction': 16046}, {'date': datetime.datetime(2020, 5, 7, 0, 0), 'capacity': 28017198, 'turnover': 8321170642, 'open': 294.5, 'high': 299.0, 'low': 294.5, 'close': 297.5, 'change': 1.5, 'transaction': 13149}, {'date': datetime.datetime(2020, 5, 8, 0, 0), 'capacity': 35613893, 'turnover': 10619091212, 'open': 300.0, 'high': 300.0, 'low': 296.0, 'close': 297.5, 'change': 0.0, 'transaction':